In [2]:
# models_research_report_structured.py
from __future__ import annotations
from typing import List, Optional
from pydantic import BaseModel, Field,ConfigDict

# ---------- META ----------

class Analyst(BaseModel):
    name: str = Field(..., description="Tên chuyên viên phân tích")
    email: Optional[str] = Field(None, description="Email liên hệ (nếu có)")
    role: Optional[str] = Field(None, description="Vai trò: Lead, Junior,...")

class Meta(BaseModel):
    ticker: str = Field(..., description="Mã cổ phiếu, ví dụ: MSH")
    company_name: Optional[str] = Field(
        None, description="Tên công ty, ví dụ: CTCP May Sông Hồng"
    )
    exchange: Optional[str] = Field(
        None, description="Sàn niêm yết, ví dụ: HOSE, HNX, UPCOM"
    )
    sector: Optional[str] = Field(
        None, description="Ngành, ví dụ: Dệt may"
    )
    industry: Optional[str] = Field(
        None, description="Tiểu ngành / phân ngành chi tiết (nếu có)"
    )
    broker: Optional[str] = Field(
        None, description="Tên công ty chứng khoán phát hành báo cáo"
    )
    report_type: Optional[str] = Field(
        None,
        description="Loại báo cáo: Initiation, Company Update, Result Flash, Sector,...",
    )
    language: Optional[str] = Field("vi", description="Ngôn ngữ của báo cáo, vi/en")
    publish_date: Optional[str] = Field(
        None,
        description="Ngày phát hành báo cáo, định dạng YYYY-MM-DD nếu parse được",
    )
    analysts: List[Analyst] = Field(
        default_factory=list, description="Danh sách chuyên viên phụ trách báo cáo"
    )
    source_file: Optional[str] = Field(
        None, description="Tên file nguồn PDF hoặc path tương đối, nếu muốn lưu lại"
    )
    source_pages: Optional[str] = Field(
        None, description="Thông tin trang trích xuất (ví dụ: '1-12' hoặc '2,4-7')"
    )
    report_id: Optional[int] = Field(
        None, description="id để tải nội dung report"
    )

# ---------- RECOMMENDATION (BẢN GỌN, KHÔNG dict) ----------

class RecommendationItem(BaseModel):
    rating: str = Field(
        ...,
        description="Khuyến nghị chính: ví dụ BUY/HOLD/SELL hoặc MUA/GIỮ/BÁN."
    )
    target_price: Optional[float] = Field(
        None, description="Giá mục tiêu (VND/cp)"
    )
    upside_pct: Optional[float] = Field(
        None, description="Mức tăng/giảm kỳ vọng so với giá hiện tại, tính theo %"
    )
    time_horizon: Optional[str] = Field(
        None, description="Thời gian khuyến nghị, ví dụ: '12 tháng'"
    )
    catalysts: Optional[str] = Field(
        None, description="Các yếu tố tích cực/catalyst chính hỗ trợ luận điểm"
    )
    risks: Optional[str] = Field(
        None, description="Các rủi ro chính gắn với khuyến nghị này"
    )

class Recommendation(BaseModel):
    summary: str = Field(
        ...,
        description=(
            "Tóm tắt 1-3 câu về khuyến nghị đầu tư tổng thể: "
            "rating, target_price, upside, bối cảnh chính."
        ),
    )
    items: List[RecommendationItem] = Field(
        ...,
        description="Danh sách khuyến nghị chi tiết (thường 1 item).",
    )
    # rating_scale bỏ dict, dùng text cho nhẹ, tránh lỗi schema
    rating_scale_text: Optional[str] = Field(
        None,
        description="Nếu báo cáo có giải thích thang rating thì tóm tắt lại bằng 1 đoạn text.",
    )
    sector_view: Optional[str] = Field(
        None, description="Quan điểm chung về ngành (tích cực, trung lập, tiêu cực...)"
    )

# ---------- INVESTMENT THESIS ----------

class ThesisPillar(BaseModel):
    title: str = Field(..., description="Tiêu đề trụ cột luận điểm, ngắn gọn")
    description: str = Field(
        ...,
        description="Mô tả chi tiết trụ cột (1–3 câu): vì sao đây là driver chính.",
    )
    time_horizon: Optional[str] = Field(
        None, description="Khoảng thời gian luận điểm này có hiệu lực (nếu nêu)"
    )
    evidence: Optional[str] = Field(
        None,
        description="Các bằng chứng, số liệu, diễn giải được trích từ báo cáo",
    )
    key_metrics: Optional[List[str]] = Field(
        None,
        description="Các chỉ số định lượng chính liên quan (ví dụ: ['gross_margin', 'EPS'])",
    )

class InvestmentThesis(BaseModel):
    summary: Optional[str] = Field(
        None,
        description=(
            "Tổng quan luận điểm đầu tư (2–5 câu) – tại sao nên/không nên "
            "đầu tư cổ phiếu này theo quan điểm của CTCK."
        ),
    )
    pillars: List[ThesisPillar] = Field(
        default_factory=list,
        description="3–5 trụ cột chính của luận điểm đầu tư (nếu có thể trích ra).",
    )

# ---------- KEY POINTS ----------

class KeyPoints(BaseModel):
    positives: List[str] = Field(
        default_factory=list,
        description="Các ý tích cực nổi bật (bullet), mỗi phần tử 1 câu ngắn.",
    )
    negatives: List[str] = Field(
        default_factory=list,
        description="Các ý tiêu cực/điểm yếu chính (bullet).",
    )
    watch_list: List[str] = Field(
        default_factory=list,
        description="Các yếu tố cần theo dõi (watch-list): sự kiện, biến số vĩ mô, chính sách, v.v.",
    )

# ---------- ROOT MODEL DÙNG CHO RESPONSE_FORMAT ----------

class ResearchReportStructured(BaseModel):
    """
    Schema tối giản cho structured output:
    chỉ gồm meta, recommendation, investment_thesis, key_points.
    Các phần khác (risks, valuation, earnings_outlook, ...) để xử lý sau.
    """
    model_config=ConfigDict(
        title = "ResearchReportStructured",
        str_strip_whitespace = True,
        validate_assignment = True
    )
    meta: Meta = Field(..., description="Thông tin nhận diện báo cáo")
    recommendation: Recommendation = Field(
        ..., description="Khuyến nghị đầu tư từ CTCK"
    )
    investment_thesis: Optional[InvestmentThesis] = Field(
        None, description="Luận điểm đầu tư chính"
    )
    key_points: Optional[KeyPoints] = Field(
        None, description="Các điểm nhấn chính: tích cực/tiêu cực/watch-list"
    )




In [3]:
# prompt_research_report.py
from langchain_core.prompts import ChatPromptTemplate

RESEARCH_REPORT_PROMPT = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
Bạn là một chuyên gia phân tích chứng khoán Việt Nam.
Nhiệm vụ của bạn: ĐỌC KỸ nội dung báo cáo phân tích của công ty chứng khoán
và TRẢ VỀ một object JSON theo schema Pydantic `ResearchReport`.

RẤT QUAN TRỌNG:
- Chỉ trích xuất thông tin CÓ XUẤT HIỆN trong báo cáo.
- KHÔNG được bịa số liệu, ngày tháng, rating, target price, WACC,... nếu báo cáo không nêu.
- Nếu không chắc, hãy để field đó là null hoặc danh sách rỗng.
- Tập trung ƯU TIÊN cho 4 phần sau:
  1) meta
  2) recommendation
  3) investment_thesis
  4) key_points

Giải thích chi tiết:

1. meta
   - ticker: lấy mã cổ phiếu chính của báo cáo (vd: MSH).
   - company_name: tên đầy đủ công ty nếu có.
   - exchange: HOSE / HNX / UPCOM nếu có.
   - sector / industry: nếu báo cáo có nêu ngành/tập đoàn.
   - broker: tên công ty chứng khoán phát hành báo cáo.
   - report_type: Initiation / Company Update / Result Flash / Sector... nếu có.
   - publish_date: nếu có ngày phát hành trên báo cáo, hãy chuẩn hóa về dạng YYYY-MM-DD nếu có thể. Định dạng ngày Việt Nam có dạng DD/MM/YYYY hoặc DD-MM-YYYY, nếu thấy MM>12 thì có nghĩa đang định dạng là MM/DD/YYYY
   - analysts: danh sách chuyên viên (tên, email, role nếu có).
   - source_file: giữ nguyên tên file được truyền vào.
   - source_pages: bạn có thể để null (sẽ xử lý sau bằng pipeline khác).
   - report_id: id để lấy nội dung report
2. recommendation
   - summary: tóm tắt 1–3 câu, phải gồm: rating, target_price, upside (%), thời gian (nếu có),
     và 1–2 ý về bối cảnh kết quả kinh doanh/góc nhìn chính.
   - items: thường chỉ 1 phần tử. Mỗi item:
       + rating: đúng như báo cáo (BUY/HOLD/SELL hoặc MUA/GIỮ/BÁN...). Không tự dịch sang tiếng khác.
       + target_price: số giá mục tiêu (VND/cp), nếu có.
       + upside_pct: phần trăm tăng/giảm so với giá hiện tại, nếu có.
       + time_horizon: ví dụ '12 tháng' nếu có.
       + catalysts: tóm tắt các yếu tố tích cực chính.
       + risks: tóm tắt các rủi ro chính liên quan tới khuyến nghị.
   - rating_scale, sector_view: CHỈ điền nếu báo cáo có nêu rõ; nếu không thì để null.

3. investment_thesis
   - summary: 2–5 câu tóm tắt luận điểm đầu tư tổng thể của báo cáo
     (vì sao nên/không nên đầu tư cổ phiếu này).
   - pillars: cố gắng rút ra 3–5 "trụ cột" rõ ràng, mỗi pillar:
       + title: 1 câu ngắn (ví dụ: "Biên LNG cải thiện nhờ đơn hàng CMT").
       + description: 1–3 câu giải thích cụ thể.
       + time_horizon: nếu tồn tại trong báo cáo (ngắn hạn/trung hạn/dài hạn hoặc năm cụ thể).
       + evidence: nếu có thể, trích dẫn ngắn gọn dữ liệu/sự kiện từ báo cáo.
       + key_metrics: danh sách tên chỉ số nếu trong phần này có đề cập (vd: ["gross_margin", "EPS"]).
   - Nếu báo cáo không có cấu trúc rõ ràng, hãy tự nhóm ý chính thành pillars logic
     nhưng KHÔNG được bịa thông tin mới.

4. key_points
   - positives: list các bullet "tích cực" (mỗi phần tử là 1 câu).
   - negatives: list các bullet "tiêu cực/hạn chế" (mỗi phần tử là 1 câu).
   - watch_list: list các yếu tố cần theo dõi (sự kiện, rủi ro, biến số vĩ mô, thuế quan, tỷ giá...).
   - Các bullet nên ngắn gọn, rõ ý, trích từ hoặc tổng hợp trực tiếp từ nội dung báo cáo.

Các phần khác trong schema (risks, valuation, earnings_outlook, business_profile, events_and_timeline):
- Bạn CHỈ điền nếu thông tin xuất hiện rõ ràng trong báo cáo.
- Nếu không chắc, hoặc không có dữ liệu, hãy để None / null / danh sách rỗng.

Đầu ra cuối cùng PHẢI là một JSON object khớp 100% với schema Pydantic ResearchReport.
Không thêm field lạ, không bớt field, không kèm giải thích bên ngoài JSON.

""",
        ),
        (
            "user",
            """
Bạn được cung cấp:
- Tên file báo cáo: {source_file}
- report_id: {report_id}
- Nội dung text đã được trích xuất từ PDF (có thể xuống dòng, có lỗi OCR nhẹ).

Nhiệm vụ: dựa trên nội dung này, hãy sinh ra object ResearchReport đúng schema.

--- BẮT ĐẦU NỘI DUNG BÁO CÁO ---
{report_text}
--- KẾT THÚC NỘI DUNG BÁO CÁO ---
""",
        ),
    ]
)


In [4]:
# parse_msh_report.py
from pathlib import Path
from typing import Optional
import requests
from pypdf import PdfReader
from langchain_openai import ChatOpenAI
from Class.auth import FireAnt
from sqlalchemy.orm import Session
from manage_engine import engine_stockDB
from Model.model_infoMarket import Report
from io import BytesIO
import re
def download_report_pdf_bytes(report_id: int, engine=None):
    """
    Tải PDF FireAnt → trả về (pdf_bytes, filename)
    filename có thể None nếu FireAnt không trả về
    """
    if engine is None:
        engine = engine_stockDB

    # --- Lấy real_id ---
    with Session(engine) as db:
        row = db.query(
            Report.report_id,
            Report.symbol,
            Report.date
            ).filter(Report.report_id == report_id,Report.symbol is not None,Report.date>='2025-01-01').first()
        if not row:
            raise ValueError(f"Không tìm thấy report id={report_id}")
        real_id = str(row[0])

    # --- Gọi FireAnt ---
    url = f"https://static.fireant.vn/reports/content/{real_id}"
    try:
        fa = FireAnt()
        headers = fa.headers()
        r = requests.get(url, headers=headers, timeout=20)

        if r.status_code != 200:
            raise RuntimeError(f"Tải file thất bại, code={r.status_code}")

        # --- Lấy filename từ Content-Disposition ---
        filename = None
        cd = r.headers.get("content-disposition")
        if cd:
            match = re.search(r'filename="?([^"]+)"?', cd)
            if match:
                filename = match.group(1)

        # fallback nếu không có header filename:
        if filename is None:
            filename = f"{report_id}.pdf"

        return r.content, filename

    except Exception as e:
        raise RuntimeError(f"Lỗi tải FireAnt PDF: {e}") from e




def extract_pdf_text_from_bytes(pdf_bytes: bytes, max_pages: Optional[int] = None) -> str:
    """
    Đọc PDF từ bytes và nối text theo trang.
    """
    reader = PdfReader(BytesIO(pdf_bytes))
    texts = []

    for i, page in enumerate(reader.pages):
        if max_pages is not None and i >= max_pages:
            break
        txt = page.extract_text() or ""
        texts.append(f"\n===== PAGE {i+1} =====\n{txt}")

    return "\n".join(texts)


def parse_research_report_from_pdf(
    report_id=int,
    model_name: str = "gpt-4.1-mini",  # hoặc "gpt-4o-mini" tuỳ bạn
    temperature: float = 0.0,
    max_pages: Optional[int] = 15,
) -> ResearchReportStructured:
    # 1. Extract text
    pdf_bytes,filename=download_report_pdf_bytes(report_id)
    report_text = extract_pdf_text_from_bytes(pdf_bytes, max_pages=max_pages)
    source_file = filename if filename is None else f'REPORT_ID_{report_id}'

    # 2. Khởi tạo LLM + structured output
    llm = ChatOpenAI(model=model_name, temperature=temperature)
    structured_llm = llm.with_structured_output(ResearchReportStructured)

    # 3. Tạo chain: prompt -> structured_llm
    chain = RESEARCH_REPORT_PROMPT | structured_llm

    # 4. Gọi chain
    result: ResearchReportStructured = chain.invoke(
        {
            "source_file": source_file,
            "report_id":report_id,
            "report_text": report_text,
        }
    )
    return result


# if __name__ == "__main__":
#     pdf_path = "PVT.pdf"  # đổi lại path trong hệ thống của bạn

#     report = parse_research_report_from_pdf(pdf_path)
#     # debug nhẹ:
#     print("Ticker:", report.meta.ticker)
#     print("Broker:", report.meta.broker)
#     print("Rating:", report.recommendation.items[0].rating)
#     print("Target price:", report.recommendation.items[0].target_price)
#     print("Thesis summary:", report.investment_thesis.summary if report.investment_thesis else None)
#     print("Positives:", report.key_points.positives if report.key_points else None)


In [ ]:
RENDER_MARKDOWN_PROMPT = """
Bạn là chuyên gia phân tích chứng khoán. 
Nhiệm vụ: Dựa vào dữ liệu JSON bên dưới (đúng theo schema ResearchReportStructured),
hãy tạo báo cáo phân tích dạng MARKDOWN theo TEMPLATE chuẩn.

YÊU CẦU:
- KHÔNG thêm thông tin không có trong JSON.
- KHÔNG phóng đại, không suy diễn.
- Nếu thiếu dữ liệu, bỏ phần viết về dữ liệu đó.
- Văn phong: Học thuật – chuyên nghiệp – cô đọng.
- Format phải giống mẫu PVT.

--- JSON REPORT ---
{json_report}
--- END JSON REPORT ---

--- TEMPLATE MARKDOWN ---
🔎 **BÁO CÁO PHÂN TÍCH – {{TICKER}} ({{COMPANY_NAME}})**  
**Khuyến nghị:** {{RATING}} • **Giá mục tiêu:** {{TARGET_PRICE}} VNĐ/cp • **Upside:** {{UPSIDE}}% - {{publish_date}} (chuyển sang format %d/%m/%Y)

---

# 1. Kết luận đầu tư
{{THESIS_SUMMARY}}

---

# 2. Khuyến nghị & Giá mục tiêu

## 2.1. Khuyến nghị {{RATING}}
- **Giá mục tiêu:** {{TARGET_PRICE}} VNĐ/cp  
- **Upside 12 tháng:** {{UPSIDE}}%  
- **Đánh giá:** {{RECOMMENDATION_SUMMARY}}

## 2.2. Yếu tố hỗ trợ
{{POSITIVE_BULLETS}}

## 2.3. Rủi ro
{{NEGATIVE_BULLETS}}

---

# 3. Các trụ cột đầu tư

Hãy diễn đạt lại mỗi trụ cột đầu tư thành **một đoạn văn trôi chảy**, gồm:
- Ý chính của pillar (title)
- Diễn giải (description)
- Gài tự nhiên thông tin dạng bằng chứng/evidence (nếu có)
- Không nhắc tới từ “bằng chứng”, “chỉ số chính”, “thời hạn”
- Không liệt kê key_metrics
- Viết như một đoạn bình luận phân tích của chuyên viên

Định dạng mỗi trụ cột:
## 3.{{pillar_index}}. {{title}}
{{paragraph}}

Sau đây là danh sách trụ cột cần viết lại theo dạng văn mạch lạc:
{{PILLARS_SECTION}}

---

# 4. Các điểm nhấn tích cực & tiêu cực

## 4.1. 🟢 Điểm tích cực
{{POSITIVES}}

## 4.2. 🔴 Điểm tiêu cực
{{NEGATIVES}}

## 4.3. Danh mục theo dõi
{{WATCH_LIST}}

---

# 5. Kết luận
{{FINAL_CONCLUSION}}
--- END TEMPLATE ---

Hãy thay thế {{...}} bằng nội dung phù hợp từ JSON. 
Trả về DUY NHẤT markdown.
"""


In [6]:
import json
from langchain_openai import ChatOpenAI

def render_markdown_from_report(report_structured):
    llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.2)

    json_report = json.dumps(report_structured, ensure_ascii=False, indent=2) \
        if isinstance(report_structured, dict) \
        else json.dumps(report_structured.model_dump(), ensure_ascii=False, indent=2)

    prompt = RENDER_MARKDOWN_PROMPT.format(json_report=json_report)
    resp = llm.invoke(prompt)
    return resp.content


In [7]:
from Model.model_infoMarket import ResearchReport,SessionLocal
from manage_engine import engine_stockDB
from datetime import datetime, date
from concurrent.futures import ThreadPoolExecutor
with Session(engine_stockDB) as db:
    rows = db.query(
        Report.report_id,
        Report.symbol,
        Report.date
    ).filter(
        Report.symbol.is_not(None),
        Report.date >= '2025-01-01'
    ).order_by(Report.date.desc()).all()
    report_ids=db.query(ResearchReport.report_id).all()
    rrIds=[id[0] for id in report_ids]
  
def llm_research(report_id,ticker_db,publish_date_db):
    # if str(report_id) in rrIds:
    #     return
    result=parse_research_report_from_pdf(report_id)
    # Code để insert/update
    session = SessionLocal()
    try:
        # Chuẩn bị dữ liệu
        if result.meta.ticker=='' or  result.meta.ticker is None:
            return
        ticker = result.meta.ticker
        if result.meta.publish_date:
            if datetime.strptime(result.meta.publish_date,"%Y-%m-%d")>=publish_date_db:
                publish_date = result.meta.publish_date 
            else:
                publish_date=publish_date_db.strftime("%Y-%m-%d")
                result.meta.publish_date =publish_date
        else:
            publish_date=publish_date_db.strftime("%Y-%m-%d")
            result.meta.publish_date =publish_date
        
        if len(result.recommendation.items) > 0:
            rating = result.recommendation.items[0].rating.upper()
            target_price = result.recommendation.items[0].target_price
        else:
            rating = None 
            target_price = None
        
        content_json = result.model_dump()
        content=render_markdown_from_report(result)
        # Tạo object ORM
        if isinstance(publish_date, str):
            publish_date = datetime.strptime(publish_date, '%Y-%m-%d').date()
        elif isinstance(publish_date, datetime):
            publish_date = publish_date.date()
        report = ResearchReport(
            report_id=str(report_id),
            ticker=ticker,
            publish_date=publish_date,
            rating=rating,
            target_price=target_price,
            content_json=content_json,
            content=content
        )
        
        # Insert hoặc Update (merge sẽ tự động check)
        session.merge(report)  # merge sẽ update nếu tồn tại, insert nếu chưa có
        session.commit()
        print("Done:",report_id,ticker)
    except Exception as e:
        session.rollback()
        print(f"Error: {e}")
    finally:
        session.close()
with ThreadPoolExecutor(max_workers=8) as executor:
    for report_id,ticker_db,publish_date_db in rows:
        executor.submit(llm_research,report_id,ticker_db,publish_date_db )

Done: 77520 PVB
Done: 77533 QNS
Done: 77522 CTG
Done: 77521 HPG
Done: 77519 CTR
Done: 77523 BSR
Done: 77528 BMP
Done: 78176 NT2
Done: 76276 TCB
Done: 77529 PVD
Done: 77530 CTG
Done: 76275 DCM
Done: 75683 VCI
Done: 76278 PC1
Done: 76279 HPG
Done: 76280 VCB
Done: 75681 MWG
Done: 77524 KBC
Done: 77526 FMC
Done: 77531 SCS
Done: 77527 PVS
Done: 75682 GMD
Done: 75680 GEG
Done: 74990 FPT
Done: 77525 IDC
Done: 74997 VHC
Done: 74998 HHV
Done: 74989 NKG
Done: 74999 KBC
Done: 75000 SZC
Done: 75001 IDC
Done: 74994 GAS
Done: 74991 MWG
Done: 74993 PVT
Done: 75677 VNM
Done: 75676 PPH
Done: 74104 null
Done: 75002 VPB
Done: 74995 SAB
Done: 74996 CTD
Done: 74103 MSH
Done: 75003 POW
Done: 74110 null
Done: 74083 PHR
Done: 74046 MSH
Done: 74101 TCB
Done: 74084 BAF
Done: 74085 HAH
Done: 74102 VNM
Done: 74045 PHR
Done: 74105 NLG
Done: 74071 KBC
Done: 74047 VCB
Done: 74109 POW
Done: 74078 MBB
Done: 74062 HPG
Done: 74081 FRT
Done: 74100 ANV
Done: 74048 GMD
Done: 74106 BSR
Done: 74080 SIP
Done: 74049 QNS
Done: 

Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 37 0 (offset 0)
Ignoring wrong pointing object 41 0 (offset 0)
Ignoring wrong pointing object 43 0 (offset 0)
Ignoring wrong pointing object 53 0 (offset 0)


Done: 74067 REE
Done: 74092 PC1
Done: 74093 DGC
Done: 74090 TCH
Done: 74091 ACB
Done: 74094 PDR
Done: 74095 ACB
Done: 74089 REE
Done: 74034 VND
Done: 74073 FPT
Done: 74087 IJC
Done: 74035 MBB
Done: 74033 DPM
Done: 74036 DGC
Done: 74072 HT1
Done: 74074 IMP
Done: 74042 TCB
Done: 74037 MBB
Done: 74040 PVS
Done: 74039 MCH
Done: 74004 KBC
Done: 74043 HVN
Done: 74014 FMC
Done: 74032 VPB
Done: 74005 null
Done: 74002 TNG
Done: 74003 VPB
Done: 74029 BMP
Done: 74022 FPT
Done: 74030 HPG
Done: 74028 CTD
Done: 74021 NT2
Done: 73997 DBD
Done: 74008 ACV
Done: 74010 ANV
Done: 74011 PVD
Done: 73535 HDG
Done: 73998 HVN
Done: 72447 IDC
Done: 70723 DGW
Done: 74009 ACV
Done: 72442 HPG
Done: 70716 CTD
Done: 73536 CTG
Done: 73538 DCM
Done: 70719 VTP
Done: 70082 MWG
Done: 70724 MSN
Done: 69861 MSH
Done: 70084 HVN
Done: 70720 ACB
Done: 69855 null
Done: 70722 DCM
Done: 70721 DCM
Done: 69864 ACB
Done: 69856 VPB
Done: 69850 SCS
Done: 69867 TCM
Done: 69858 DBC
Done: 69838 HHV
Done: 69851 CTD
Done: 69837 IJC
Done: 

Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 30 0 (offset 0)
Ignoring wrong pointing object 32 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 71 0 (offset 0)


Done: 68140 GMD
Done: 68137 SZC
Done: 68135 OCB
Done: 68133 PLC
Done: 67383 SIP
Done: 67838 PNJ
Done: 67384 HSG
Done: 67389 PC1
Done: 66966 DXG
Done: 66960 ANV
Done: 66961 DGW
Done: 66965 FPT
Done: 66962 VCB
Done: 66959 PPH
Done: 66963 MSH
Done: 66416 ANV
Done: 66830 HPG
Done: 66268 NAB
Done: 66264 PDR
Done: 66267 ACB
Done: 66265 HAH
Done: 66269 IDC
Done: 66417 VTP
Done: 65840 IDC
Done: 65844 HSG
Done: 65842 HDB
Done: 65843 AST
Done: 65841 MSB
Done: 65845 DCM
Done: 65846 DCM
Done: 65535 SZC
Done: 65536 MSB
Done: 65537 NKG
Done: 66822 HDG
Done: 66825 SAB
Done: 65534 KBC
Done: 65417 DXG
Done: 65416 FPT
Done: 66829 MWG
Done: 66828 FMC
Done: 66820 GMD
Done: 65419 SZC
Done: 66823 VHC
Done: 65420 CMG
Done: 65424 NLG
Done: 66824 HSG
Done: 65418 HAH
Done: 66827 DPM
Done: 65422 VNM
Done: 65412 GEG
Done: 65413 CSV
Done: 65415 SAB
Done: 65414 GAS
Done: 65405 NKG
Done: 65407 MSH
Done: 65401 SIP
Done: 65411 VHC
Done: 69820 TCM
Done: 65402 VIB
Done: 65395 HSG
Done: 65394 PVD
Done: 65396 VNM
Done: 65

...

Done: 65320 SZC
Done: 65269 TRC
Done: 65345 VPB
Done: 65295 FPT
Done: 65270 BMP
Done: 65293 DBC
Done: 65294 VGC
Done: 65272 MBS
Done: 65257 FPT
Done: 65258 HT1
Done: 65255 FPT
Done: 65262 GEE
Done: 65271 DRC
Done: 65259 LAS
Done: 65263 FRT
Done: 65260 VCG
Done: 65261 GMD
Done: 65250 TNG
Done: 65236 BWE
Done: 65230 HAG
Done: 65235 ACV


Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 30 0 (offset 0)
Ignoring wrong pointing object 32 0 (offset 0)
Ignoring wrong pointing object 44 0 (offset 0)
Ignoring wrong pointing object 61 0 (offset 0)
Ignoring wrong pointing object 73 0 (offset 0)
Ignoring wrong pointing object 78 0 (offset 0)
Ignoring wrong pointing object 80 0 (offset 0)


Done: 65232 KBC
Done: 65247 DCM
Done: 65243 IDC
Done: 65231 DCM
Done: 65233 BMP
Done: 65226 BID
Done: 65223 DGW
Done: 65296 VGC
Done: 65234 BAF
Done: 65210 GEG
Done: 65216 VIB
Done: 65214 VPB
Done: 65199 VIB
Done: 65211 DXG
Done: 65198 BMP
Done: 65222 VEA
Done: 65205 HAX
Done: 65204 POW
Done: 65186 LCG
Done: 65200 HCM
Done: 65201 HDG
Done: 65188 SAB
Done: 65189 ANV
Done: 65187 HAH
Done: 65194 SSI
Done: 65193 GAS
Done: 65190 HAH
Done: 65174 ACV
Done: 65203 BID
Done: 65195 SIP
Done: 65176 NLG
Done: 65172 GMD
Done: 65164 null
Done: 65171 MML
Done: 65170 VPB
Done: 65162 HHV
Done: 65161 VCG
Done: 65191 MBB
Done: 65167 HHV
Done: 65160 NT2
Done: 65165 BWE
Done: 65192 TNH
Done: 65169 VIB
Done: 65155 HAX
Done: 65154 MSH
Done: 65168 HDG
Done: 65147 MWG
Done: 65151 MBB
Done: 65152 CTI
Done: 65133 SHB
Done: 65126 GVR
Done: 65131 QNS
Done: 65130 VHM
Done: 65125 VCB
Done: 65178 HSG
Done: 65122 BID
Done: 65129 STB
Done: 65150 HSG
Done: 64353 AGG
Done: 65134 HDB
Done: 65179 NVL
Done: 64365 GEG
Done: 6

Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 24 0 (offset 0)
Ignoring wrong pointing object 34 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 55 0 (offset 0)
Ignoring wrong pointing object 60 0 (offset 0)
Ignoring wrong pointing object 65 0 (offset 0)


Done: 47560 MWG
Done: 49018 HPG
Done: 49905 C4G
Done: 49906 HVN
Done: 49019 HAX
Done: 49020 TCB
Done: 47052 PVD
Done: 46998 HHV
Done: 47029 BAF
Done: 46999 IJC
Done: 47000 DPR
Done: 47031 null
Done: 47044 PVS
Done: 47004 VHC
Done: 47996 DCM
Done: 47022 DBD
Done: 47047 DGW
Done: 47048 PVT
Done: 47994 MWG
Done: 47995 NKG
Done: 47025 CTR
Done: 47032 MSH
Done: 47050 SAB
Done: 47993 SHB
Done: 46410 null
Done: 47001 SIP
Done: 47992 DXG
Done: 46958 HPG
Done: 47049 PNJ
Done: 47026 VPB
Done: 47045 BCM
Done: 47036 VGS
Done: 47035 BID
Done: 46409 FPT
Done: 46407 GMD
Done: 46405 HDG
Done: 47037 MSB
Done: 45814 IDC
Done: 46411 PNJ
Done: 47027 REE
Done: 46408 HPG
Done: 47042 CTI
Done: 43650 MSH
Done: 46406 BSR
Done: 43485 null
Done: 44782 PVS
Done: 44023 MCH
Done: 46412 TCB
Done: 43646 VNM
Done: 43503 MSH
Done: 43518 CTD
Done: 43519 HHV
Done: 43486 FRT
Done: 43483 DPM
Done: 43487 DBC
Done: 43484 SIP
Done: 43490 DGW
Done: 43473 POW
Done: 43489 VCS
Done: 43474 NLG
Done: 43471 JU
Done: 43500 VTP
Done: 

In [8]:
report_id

43398

🔎 **BÁO CÁO PHÂN TÍCH – BSR (CTCP Lọc – Hóa dầu Bình Sơn)**  
**Khuyến nghị:** Tăng tỷ trọng • **Giá mục tiêu:** 18.200 VNĐ/cp • **Upside:** 13,7% - 14/11/2025

---

# 1. Kết luận đầu tư
Lọc – Hóa dầu Bình Sơn có lợi thế từ biên lợi nhuận lọc dầu cải thiện nhờ giá dầu thô ổn định và sản lượng tiêu thụ tăng. Nhu cầu tiêu thụ sản phẩm hóa dầu trong nước và khu vực dự kiến tăng mạnh, trong khi dự án nâng cấp mở rộng nhà máy Dung Quất sẽ là động lực tăng trưởng dài hạn. Công ty đã hoàn thành vượt kế hoạch lợi nhuận 9 tháng đầu năm 2025, cho thấy hiệu quả hoạt động tích cực.

---

# 2. Khuyến nghị & Giá mục tiêu

## 2.1. Khuyến nghị Tăng tỷ trọng
- **Giá mục tiêu:** 18.200 VNĐ/cp  
- **Upside 12 tháng:** 13,7%  
- **Đánh giá:** Khuyến nghị Tăng tỷ trọng cổ phiếu BSR với giá mục tiêu 18.200 đồng/cp, upside 13,7% trong bối cảnh lợi nhuận Quý 3/2025 dương trở lại nhờ biên lợi nhuận lọc dầu cải thiện và kỳ vọng biên lợi nhuận tiếp tục phục hồi trong Quý 4/2025.

## 2.2. Yếu tố hỗ trợ
- Lợi nhuận Quý 3/2025 dương trở lại với LNST 908,6 tỷ đồng, cải thiện mạnh so với cùng kỳ lỗ 1.210 tỷ đồng.  
- Lũy kế 9 tháng đầu năm 2025, doanh thu tăng 19,4% và LNST tăng 219,7% so với cùng kỳ, vượt 287% kế hoạch lợi nhuận cả năm.  
- Biên lợi nhuận lọc dầu dự kiến phục hồi trong Quý 4/2025 nhờ nguồn cung toàn cầu thấp và các sự cố gián đoạn nguồn cung.  
- Nhu cầu tiêu thụ sản phẩm hóa dầu trong nước và khu vực tăng mạnh, với tổng cầu xăng dầu nội địa dự kiến tăng 7-10% mỗi năm.  
- Dự án nâng cấp mở rộng nhà máy lọc dầu Dung Quất sẽ tăng công suất và đa dạng hóa nguồn đầu vào, tạo động lực tăng trưởng dài hạn.

## 2.3. Rủi ro
- Cổ phiếu đang giao dịch trong biên độ tích lũy với động lượng tăng có phần suy yếu theo chỉ báo kỹ thuật MACD.  
- P/E hiện tại cao ở mức 123,6 lần, có thể phản ánh kỳ vọng tăng trưởng cao nhưng cũng tiềm ẩn rủi ro định giá.  
- Nguồn cung lọc hóa dầu hiện chỉ đáp ứng khoảng 70% nhu cầu nội địa, có thể gây áp lực cung cầu trong ngắn hạn.

---

# 3. Các trụ cột đầu tư

## 3.1. Biên lợi nhuận lọc dầu cải thiện  
Biên lợi nhuận lọc dầu của BSR tăng lên nhờ giá dầu thô ổn định hơn và sản lượng tiêu thụ tăng trong Quý 3/2025, với lợi nhuận sau thuế đạt 908,6 tỷ đồng so với khoản lỗ 1.210 tỷ đồng cùng kỳ năm trước. Xu hướng này được hỗ trợ bởi các báo cáo từ OPEC và EIA cho thấy biên lợi nhuận lọc dầu hồi phục từ tháng 5/2025. Dự kiến biên lợi nhuận sẽ tiếp tục phục hồi trong Quý 4/2025 do nguồn cung toàn cầu thấp và các sự cố gián đoạn nguồn cung.

## 3.2. Nhu cầu tiêu thụ sản phẩm hóa dầu tăng mạnh  
Tăng trưởng GDP dự kiến đạt 8% trong năm 2025 và duy trì mức hai chữ số trong giai đoạn 2026-2030 sẽ thúc đẩy nhu cầu tiêu thụ xăng dầu và sản phẩm hóa dầu trong nước cũng như khu vực. Hiện nguồn cung chỉ đáp ứng khoảng 70% nhu cầu nội địa, trong khi tổng cầu tiêu thụ xăng dầu nội địa dự kiến tăng 7-10% mỗi năm. Lượng khách quốc tế đến Việt Nam trong 6 tháng đầu năm 2025 cũng tăng 22%, góp phần gia tăng nhu cầu tiêu thụ sản phẩm hóa dầu.

## 3.3. Dự án nâng cấp mở rộng nhà máy lọc dầu Dung Quất  
Dự án nâng cấp mở rộng nhà máy lọc dầu Dung Quất dự kiến đi vào hoạt động năm 2028, sẽ tăng công suất lọc dầu lên 17% và đa dạng hóa nguồn đầu vào, giúp tối ưu chi phí và nâng cao lợi nhuận dài hạn cho BSR. Công ty dự kiến phát hành cổ phiếu tăng vốn trong Quý 4/2025 để tài trợ cho dự án này, đồng thời nâng cao khả năng xử lý dầu thô có hàm lượng lưu huỳnh cao.

---

# 4. Các điểm nhấn tích cực & tiêu cực

## 4.1. 🟢 Điểm tích cực
- Lợi nhuận Quý 3/2025 dương trở lại với LNST 908,6 tỷ đồng, cải thiện mạnh so với cùng kỳ lỗ 1.210 tỷ đồng.  
- Lũy kế 9 tháng đầu năm 2025, doanh thu tăng 19,4% và LNST tăng 219,7% so với cùng kỳ, vượt 287% kế hoạch lợi nhuận cả năm.  
- Biên lợi nhuận lọc dầu dự kiến phục hồi trong Quý 4/2025 nhờ nguồn cung toàn cầu thấp và các sự cố gián đoạn nguồn cung.  
- Nhu cầu tiêu thụ sản phẩm hóa dầu trong nước và khu vực tăng mạnh, với tổng cầu xăng dầu nội địa dự kiến tăng 7-10% mỗi năm.  
- Dự án nâng cấp mở rộng nhà máy lọc dầu Dung Quất sẽ tăng công suất và đa dạng hóa nguồn đầu vào, tạo động lực tăng trưởng dài hạn.

## 4.2. 🔴 Điểm tiêu cực
- Cổ phiếu đang giao dịch trong biên độ tích lũy với động lượng tăng có phần suy yếu theo chỉ báo kỹ thuật MACD.  
- P/E hiện tại cao ở mức 123,6 lần, có thể phản ánh kỳ vọng tăng trưởng cao nhưng cũng tiềm ẩn rủi ro định giá.  
- Nguồn cung lọc hóa dầu hiện chỉ đáp ứng khoảng 70% nhu cầu nội địa, có thể gây áp lực cung cầu trong ngắn hạn.

## 4.3. Danh mục theo dõi
- Diễn biến giá dầu thô và biên lợi nhuận lọc dầu trong các quý tiếp theo.  
- Tiến độ và hiệu quả dự án nâng cấp mở rộng nhà máy lọc dầu Dung Quất.  
- Tác động của các sự cố gián đoạn nguồn cung và bảo dưỡng nhà máy lọc dầu tại châu Á.  
- Tăng trưởng nhu cầu tiêu thụ sản phẩm hóa dầu trong nước và khu vực.  
- Biến động thị trường chứng khoán và động thái giao dịch của khối ngoại.

---

# 5. Kết luận
CTCP Lọc – Hóa dầu Bình Sơn đang trong giai đoạn phục hồi lợi nhuận với các yếu tố hỗ trợ từ biên lợi nhuận lọc dầu cải thiện và nhu cầu thị trường tăng trưởng tích cực. Dự án nâng cấp mở rộng nhà máy Dung Quất là động lực tăng trưởng dài hạn quan trọng. Tuy nhiên, nhà đầu tư cần lưu ý các rủi ro về định giá cao và áp lực cung cầu trong ngắn hạn. Khuyến nghị tăng tỷ trọng với giá mục tiêu 18.200 VNĐ/cp, tương ứng upside 13,7%.